
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week=20190517
path = f'/opt/hawkeye/data/{this_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].sum()/len(df),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')

print(f'Total stock number = {len(df)}')

# remove bottom volume and bottom price
volume_threshold = df['volume_mean'].quantile(0.1)
df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.05)]

# calculate return & risk rank
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['return_rank'] = round(df['return'].rank(pct=True) * 100, 3)
df['risk_rank'] = round(df['VaR 99% Percent'].rank(pct=True) * 100, 3)
df['volume_rank'] = round(df['volume_mean'].rank(pct=True) * 100, 3)

# drop used columns
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%','VaR 99% Percent', 'volume_mean'], inplace=True)

print(f'Total stock number after bottom volume & price removed = {len(df)}')
print('Price 10%% pricetile = %s' % df['start price'].quantile(0.1))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

df.sort_values(by='return', ascending=False).head(20)

Total stock number = 1571
Total stock number after bottom volume & price removed = 962
Price 10% pricetile = 0.0761
Sim return 90% percentile = 1.0973


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
KAT,2019-05-10,0.4518,0.042825,0.317021,8.509,100.000,100.000,0.728
FTC,2019-05-10,0.1700,0.025319,0.190321,4.853,99.896,99.792,71.102
AVH,2019-05-16,0.4900,0.021652,0.075526,4.350,99.792,87.006,98.857
NSC,2019-05-17,0.6150,0.019579,0.208017,4.050,99.688,99.896,16.736
ANO,2019-05-17,4.7000,0.018749,0.063133,3.639,99.584,73.493,29.210
WGO,2019-05-17,0.1400,0.018281,0.110493,3.505,99.480,98.857,57.173
WZR,2019-05-17,0.1400,0.016131,0.090425,3.359,99.376,95.842,95.010
ATU,2019-05-16,0.3500,0.016391,0.075468,3.331,99.272,89.813,72.869
8CO,2019-05-16,0.1550,0.017100,0.074168,3.297,99.168,89.501,52.183


In [5]:
# high mean return 
df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
KAT,2019-05-10,0.4518,0.042825,0.317021,8.509,100.000,100.000,0.728
FTC,2019-05-10,0.1700,0.025319,0.190321,4.853,99.896,99.792,71.102
AVH,2019-05-16,0.4900,0.021652,0.075526,4.350,99.792,87.006,98.857
NSC,2019-05-17,0.6150,0.019579,0.208017,4.050,99.688,99.896,16.736
ANO,2019-05-17,4.7000,0.018749,0.063133,3.639,99.584,73.493,29.210
WGO,2019-05-17,0.1400,0.018281,0.110493,3.505,99.480,98.857,57.173
WZR,2019-05-17,0.1400,0.016131,0.090425,3.359,99.376,95.842,95.010
ATU,2019-05-16,0.3500,0.016391,0.075468,3.331,99.272,89.813,72.869
8CO,2019-05-16,0.1550,0.017100,0.074168,3.297,99.168,89.501,52.183


In [6]:
# high price
df[df['start price']>1].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
ANO,2019-05-17,4.700,0.018749,0.063133,3.639,99.584,73.493,29.210
Z1P,2019-05-17,3.520,0.013231,0.041874,2.547,98.129,49.064,87.942
5GN,2019-05-17,1.220,0.011851,0.046108,2.371,97.921,57.796,55.509
BUB,2019-05-17,1.255,0.012003,0.054048,2.220,97.713,71.933,95.738
CIA,2019-05-17,2.810,0.010121,0.040556,2.007,97.193,50.104,51.455
NEA,2019-05-17,3.830,0.009782,0.032158,1.991,96.985,38.150,90.333
MGX,2019-05-17,1.255,0.009759,0.028963,1.950,96.674,30.042,90.956
JIN,2019-05-17,17.640,0.009616,0.028292,1.883,96.362,29.730,57.069
CRD,2019-05-17,1.725,0.009541,0.043361,1.861,96.154,55.094,53.015


In [7]:
# highest price
df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
XRO,2019-05-17,61.30,0.004924,0.021471,0.936,86.694,22.245,63.825
MFG,2019-05-17,44.82,0.007133,0.018919,1.392,93.347,13.514,69.439
ALU,2019-05-17,32.61,0.005079,0.027926,0.976,87.734,36.383,67.152
CUV,2019-05-17,31.46,0.006013,0.031563,1.211,91.736,42.308,31.601
APT,2019-05-17,25.60,0.008506,0.038436,1.696,95.530,48.441,87.110
APX,2019-05-17,25.19,0.008101,0.031709,1.614,94.699,37.630,75.364
BRG,2019-05-17,18.21,0.006033,0.023060,1.158,90.852,23.493,56.861
JIN,2019-05-17,17.64,0.009616,0.028292,1.883,96.362,29.730,57.069
WEB,2019-05-17,16.80,0.004770,0.036727,0.951,87.110,52.599,68.711


In [9]:
# low risk
low_risk = df.sort_values(by='risk_rank', ascending=True).head(20).sort_values(by='return', ascending=False)
low_risk

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
CHC,2019-05-17,10.500,0.003503,0.010074,0.665,79.522,1.455,84.927
ASX,2019-05-17,76.110,0.002920,0.008799,0.561,75.364,1.040,60.395
AMC,2019-05-17,16.600,0.002735,0.007769,0.532,73.701,0.520,93.763
CMW,2019-05-17,1.185,0.002001,0.008125,0.392,64.501,0.936,94.491
WOW,2019-05-17,34.180,0.001963,0.009479,0.386,63.825,2.079,90.852
STW,2019-05-17,59.590,0.001665,0.006315,0.309,56.861,0.104,35.135
GOZ,2019-05-17,4.350,0.001604,0.008640,0.303,55.821,1.351,69.335
AVN,2019-05-17,2.270,0.001205,0.008486,0.229,50.364,1.767,58.420
EGF,2019-05-17,1.600,0.001031,0.008556,0.191,48.025,1.975,11.642


In [10]:
# high volume
high_volume = df.sort_values(by='volume_rank', ascending=False).head(20).sort_values(by='return', ascending=False)
high_volume

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
AVH,2019-05-16,0.490,0.021652,0.075526,4.350,99.792,87.006,98.857
LTR,2019-05-10,0.071,0.014446,0.090069,2.795,98.857,96.466,98.545
FEX,2019-05-10,0.078,0.013581,0.092032,2.660,98.649,97.401,98.337
FMG,2019-05-17,8.950,0.008612,0.027161,1.646,95.114,27.443,99.896
4DS,2019-05-16,0.069,0.005516,0.078144,1.042,88.669,93.971,98.233
BPT,2019-05-17,2.100,0.005080,0.025700,0.993,88.046,30.457,98.960
NEC,2019-05-17,1.880,0.003329,0.023546,0.641,78.274,27.651,99.064
MGR,2019-05-17,2.950,0.002683,0.011212,0.510,72.505,3.742,99.584
TLS,2019-05-16,3.510,0.002249,0.013454,0.439,67.100,9.667,100.000


In [12]:
# 2019-05-17 recommendations
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank
code,,,,,,,,
AVH,2019-05-16,0.49,0.021652,0.075526,4.350,99.792,87.006,98.857
ANO,2019-05-17,4.70,0.018749,0.063133,3.639,99.584,73.493,29.210
AGH,2019-05-17,0.62,0.013553,0.051464,2.718,98.753,63.825,65.385
Z1P,2019-05-17,3.52,0.013231,0.041874,2.547,98.129,49.064,87.942
FMG,2019-05-17,8.95,0.008612,0.027161,1.646,95.114,27.443,99.896
AD8,2019-05-17,7.11,0.007450,0.023410,1.438,93.867,20.790,44.906
MFG,2019-05-17,44.82,0.007133,0.018919,1.392,93.347,13.514,69.439
